# 中文语料分析报告 📊

本笔记本用于分析个人微信公众号语料的词频、关键词和可视化趋势。

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置图表样式
sns.set_style("whitegrid")
plt.style.use('seaborn-v0_8')

# 添加源代码路径
sys.path.append('../src')

print("📚 环境设置完成")

## 📊 数据加载

In [ ]:
# 读取分析结果
output_dir = '../out'

# 检查文件是否存在
files_to_check = [
    'freq_overall.csv',
    'freq_by_year.csv', 
    'tfidf_topk_by_year.csv'
]

existing_files = []
for file in files_to_check:
    file_path = os.path.join(output_dir, file)
    if os.path.exists(file_path):
        existing_files.append(file)
        print(f"✅ 找到文件: {file}")
    else:
        print(f"❌ 文件不存在: {file}")

if not existing_files:
    print("\n⚠️ 请先运行 CLI 工具生成分析结果")
    print("运行命令: python -m analysis.src.cli --root Wechat-Backup/文不加点的张衔瑜")

In [ ]:
# 加载数据
freq_overall = pd.DataFrame()
freq_by_year = pd.DataFrame()
tfidf_by_year = pd.DataFrame()

if 'freq_overall.csv' in existing_files:
    freq_overall = pd.read_csv(os.path.join(output_dir, 'freq_overall.csv'))
    print(f"📊 整体词频数据: {len(freq_overall)} 个词汇")

if 'freq_by_year.csv' in existing_files:
    freq_by_year = pd.read_csv(os.path.join(output_dir, 'freq_by_year.csv'))
    years = sorted(freq_by_year['year'].unique()) if not freq_by_year.empty else []
    print(f"📅 年度词频数据: {len(years)} 年，{len(freq_by_year)} 条记录")

if 'tfidf_topk_by_year.csv' in existing_files:
    tfidf_by_year = pd.read_csv(os.path.join(output_dir, 'tfidf_topk_by_year.csv'))
    print(f"🔍 TF-IDF 数据: {len(tfidf_by_year)} 条记录")

## 🔥 整体词频分析

In [ ]:
if not freq_overall.empty:
    # Top 20 词汇条形图
    plt.figure(figsize=(12, 8))
    top_20 = freq_overall.head(20)
    
    bars = plt.barh(range(len(top_20)), top_20['freq'], 
                    color=plt.cm.viridis(np.linspace(0, 1, len(top_20))))
    
    plt.yticks(range(len(top_20)), top_20['word'])
    plt.xlabel('词频')
    plt.title('Top 20 高频词汇', fontsize=16, fontweight='bold')
    plt.gca().invert_yaxis()
    
    # 添加数值标签
    for i, bar in enumerate(bars):
        width = bar.get_width()
        plt.text(width + width*0.01, bar.get_y() + bar.get_height()/2, 
                f'{int(width):,}', ha='left', va='center')
    
    plt.tight_layout()
    plt.show()
    
    # 统计摘要
    total_words = len(freq_overall)
    total_freq = freq_overall['freq'].sum()
    top_10_freq = freq_overall.head(10)['freq'].sum()
    
    print(f"\n📈 整体统计摘要:")
    print(f"总词汇数: {total_words:,}")
    print(f"总词频: {total_freq:,}")
    print(f"Top 10 词汇占比: {top_10_freq/total_freq:.1%}")
    print(f"词汇多样性 (词汇数/总词频): {total_words/total_freq:.4f}")
else:
    print("❌ 没有整体词频数据")

## 📅 年度趋势分析

In [ ]:
if not freq_by_year.empty:
    # 年度词汇量趋势
    yearly_stats = freq_by_year.groupby('year').agg({
        'word': 'nunique',  # 词汇数
        'freq': 'sum'       # 总词频
    }).rename(columns={'word': 'unique_words', 'freq': 'total_freq'})
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 年度词汇数变化
    ax1.plot(yearly_stats.index, yearly_stats['unique_words'], 
             marker='o', linewidth=2, markersize=8, color='#2E86AB')
    ax1.set_title('年度词汇数变化', fontweight='bold')
    ax1.set_xlabel('年份')
    ax1.set_ylabel('词汇数')
    ax1.grid(True, alpha=0.3)
    
    # 年度总词频变化
    ax2.plot(yearly_stats.index, yearly_stats['total_freq'], 
             marker='s', linewidth=2, markersize=8, color='#A23B72')
    ax2.set_title('年度总词频变化', fontweight='bold')
    ax2.set_xlabel('年份')
    ax2.set_ylabel('总词频')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 显示数据表
    yearly_stats['diversity'] = yearly_stats['unique_words'] / yearly_stats['total_freq']
    print("\n📊 年度统计:")
    print(yearly_stats.round(4))
else:
    print("❌ 没有年度词频数据")

In [ ]:
if not freq_by_year.empty:
    # 选择几个有趣的词汇分析其年度变化
    # 自动选择在多个年份都出现的高频词
    word_year_counts = freq_by_year.groupby('word')['year'].nunique().sort_values(ascending=False)
    multi_year_words = word_year_counts[word_year_counts >= 3].index[:8]  # 至少出现3年的前8个词
    
    if len(multi_year_words) > 0:
        plt.figure(figsize=(14, 8))
        
        colors = plt.cm.Set3(np.linspace(0, 1, len(multi_year_words)))
        
        for i, word in enumerate(multi_year_words):
            word_data = freq_by_year[freq_by_year['word'] == word]
            plt.plot(word_data['year'], word_data['freq'], 
                    marker='o', label=word, linewidth=2, color=colors[i])
        
        plt.title('关键词汇年度变化趋势', fontsize=16, fontweight='bold')
        plt.xlabel('年份')
        plt.ylabel('词频')
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        print(f"\n🔍 分析了 {len(multi_year_words)} 个多年度出现的关键词")
    else:
        print("❌ 没有找到多年度出现的词汇")

## 🎯 TF-IDF 关键词分析

In [ ]:
if not tfidf_by_year.empty:
    # 每年Top 10 TF-IDF关键词热力图
    years = sorted(tfidf_by_year['year'].unique())
    
    # 获取每年前10个关键词
    top_words_by_year = {}
    for year in years:
        year_data = tfidf_by_year[tfidf_by_year['year'] == year].head(10)
        top_words_by_year[year] = list(year_data['word'])
    
    # 创建词汇-年份矩阵
    all_top_words = set()
    for words in top_words_by_year.values():
        all_top_words.update(words)
    
    # 构建热力图数据
    matrix_data = []
    word_labels = []
    
    for word in all_top_words:
        row = []
        for year in years:
            if word in top_words_by_year[year]:
                # 获取该词在该年的排名（倒序，排名越高值越大）
                rank = top_words_by_year[year].index(word)
                row.append(10 - rank)  # 转换为正向权重
            else:
                row.append(0)
        matrix_data.append(row)
        word_labels.append(word)
    
    if matrix_data:
        plt.figure(figsize=(12, min(20, len(word_labels))))
        
        # 只显示至少在2年出现的词汇
        filtered_data = []
        filtered_labels = []
        for i, row in enumerate(matrix_data):
            if sum(1 for x in row if x > 0) >= 2:  # 至少在2年出现
                filtered_data.append(row)
                filtered_labels.append(word_labels[i])
        
        if filtered_data:
            sns.heatmap(filtered_data, 
                       xticklabels=years,
                       yticklabels=filtered_labels,
                       cmap='YlOrRd',
                       annot=True,
                       fmt='d',
                       cbar_kws={'label': '关键词重要性 (10=最重要)'})
            
            plt.title('年度 TF-IDF 关键词热力图', fontsize=16, fontweight='bold')
            plt.xlabel('年份')
            plt.ylabel('关键词')
            plt.tight_layout()
            plt.show()
        else:
            print("❌ 没有找到多年度的关键词")
    
    # 显示每年的Top 5关键词
    print("\n🎯 各年度 Top 5 TF-IDF 关键词:")
    for year in years:
        year_data = tfidf_by_year[tfidf_by_year['year'] == year].head(5)
        words = ', '.join(year_data['word'].tolist())
        print(f"{year}: {words}")
else:
    print("❌ 没有 TF-IDF 数据")

## 🎨 词云展示

In [ ]:
from IPython.display import Image, display
import matplotlib.image as mpimg

# 显示整体词云
overall_wordcloud_path = os.path.join(output_dir, 'wordcloud_overall.png')
if os.path.exists(overall_wordcloud_path):
    print("🎨 整体词云:")
    img = mpimg.imread(overall_wordcloud_path)
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title('整体词云', fontsize=16, fontweight='bold')
    plt.show()
else:
    print("❌ 整体词云文件不存在")

In [ ]:
# 显示年度词云
if not freq_by_year.empty:
    years = sorted(freq_by_year['year'].unique())
    
    # 计算子图布局
    n_years = len(years)
    cols = min(3, n_years)
    rows = (n_years + cols - 1) // cols
    
    if rows > 0:
        fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
        
        if n_years == 1:
            axes = [axes]
        elif rows == 1:
            axes = axes if hasattr(axes, '__len__') else [axes]
        else:
            axes = axes.flatten()
        
        found_any = False
        
        for i, year in enumerate(years):
            wordcloud_path = os.path.join(output_dir, f'wordcloud_{year}.png')
            
            if os.path.exists(wordcloud_path):
                img = mpimg.imread(wordcloud_path)
                axes[i].imshow(img)
                axes[i].set_title(f'{year} 年词云', fontweight='bold')
                axes[i].axis('off')
                found_any = True
            else:
                axes[i].text(0.5, 0.5, f'{year}\n词云文件\n不存在', 
                           ha='center', va='center', transform=axes[i].transAxes)
                axes[i].axis('off')
        
        # 隐藏多余的子图
        for i in range(n_years, len(axes)):
            axes[i].axis('off')
        
        if found_any:
            plt.suptitle('年度词云对比', fontsize=16, fontweight='bold')
            plt.tight_layout()
            plt.show()
        else:
            plt.close()
            print("❌ 没有找到年度词云文件")
else:
    print("❌ 没有年度数据")

## 💭 自我鼓励与年度口号

In [ ]:
# 生成年度口号
if not tfidf_by_year.empty:
    print("🎯 基于关键词的年度口号:")
    print("=" * 50)
    
    # 自我鼓励模板
    templates = [
        "在{year}年，我专注于{word1}，深入探索{word2}，持续思考{word3}。",
        "{year}年是{word1}之年，我在{word2}领域不断成长，对{word3}有了更深的理解。",
        "回顾{year}年，{word1}是我的关键词，通过{word2}实现突破，在{word3}方面收获满满。",
        "{year}年，我将{word1}作为主线，以{word2}为方法，向{word3}目标前进。",
        "在{year}年的旅程中，{word1}指引方向，{word2}提供动力，{word3}带来启发。"
    ]
    
    years = sorted(tfidf_by_year['year'].unique())
    
    for i, year in enumerate(years):
        year_data = tfidf_by_year[tfidf_by_year['year'] == year].head(5)
        if len(year_data) >= 3:
            words = year_data['word'].tolist()
            template = templates[i % len(templates)]
            
            slogan = template.format(
                year=year,
                word1=words[0],
                word2=words[1] if len(words) > 1 else '新领域',
                word3=words[2] if len(words) > 2 else '人生哲理'
            )
            
            print(f"\n📅 {year}年:")
            print(f"   {slogan}")
            print(f"   关键词: {', '.join(words[:5])}")
    
    print("\n" + "=" * 50)
    print("💪 总结感言:")
    
    if not freq_overall.empty:
        total_words = len(freq_overall)
        total_freq = freq_overall['freq'].sum()
        year_span = len(years)
        
        motivation = f"""通过 {year_span} 年的写作历程，我积累了 {total_words:,} 个不同的词汇，
总计 {total_freq:,} 次的表达。每一个词汇都承载着思考的痕迹，
每一次表达都见证着成长的足迹。

从数据中可以看出，我的表达越来越丰富，思考越来越深入。
这不仅是文字的积累，更是智慧的沉淀。

继续保持这份热情，让文字记录生活，让思考指引未来！ 🚀"""
        
        print(motivation)
else:
    print("❌ 没有足够的数据生成年度口号")

## 📈 Zipf 定律分析

In [ ]:
# 显示 Zipf 分析图
zipf_path = os.path.join(output_dir, 'zipf_overall.png')
if os.path.exists(zipf_path):
    print("📈 Zipf 定律分析:")
    img = mpimg.imread(zipf_path)
    plt.figure(figsize=(15, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Zipf 定律分析', fontsize=16, fontweight='bold')
    plt.show()
    
    print("\n📝 Zipf 定律解读:")
    print("""Zipf 定律表明，自然语言中词频与其排名呈反比关系。
这反映了语言使用的经济性原则：
- 少数高频词承载主要信息传递功能
- 大量低频词丰富表达的精确性和个性化
- 符合 Zipf 定律的语言使用体现了自然的平衡状态

从你的写作语料来看，这种分布特征表明你的文字表达既有核心主题，
又具备丰富的细节描述，体现了成熟的写作风格。""")
else:
    print("❌ Zipf 分析图文件不存在")

## 🔄 数据导出

In [ ]:
# 创建一个汇总的数据集用于进一步分析
if not freq_overall.empty and not freq_by_year.empty:
    # 合并数据创建完整的分析数据集
    analysis_summary = {
        'analysis_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
        'total_unique_words': len(freq_overall),
        'total_word_freq': freq_overall['freq'].sum(),
        'years_covered': sorted(freq_by_year['year'].unique()) if not freq_by_year.empty else [],
        'top_10_words': freq_overall.head(10)['word'].tolist(),
        'diversity_index': len(freq_overall) / freq_overall['freq'].sum()
    }
    
    # 保存汇总信息
    import json
    summary_path = os.path.join(output_dir, 'analysis_summary.json')
    with open(summary_path, 'w', encoding='utf-8') as f:
        json.dump(analysis_summary, f, ensure_ascii=False, indent=2)
    
    print(f"📊 分析汇总已保存到: {summary_path}")
    print("\n汇总信息:")
    for key, value in analysis_summary.items():
        if key != 'top_10_words':
            print(f"  {key}: {value}")
    print(f"  top_10_words: {', '.join(analysis_summary['top_10_words'])}")
else:
    print("❌ 数据不完整，无法生成汇总")

---

## 🎉 分析完成

本笔记本完成了对个人微信公众号语料的全面分析，包括：

1. **词频统计** - 了解最常用的词汇
2. **年度趋势** - 观察写作风格的演变
3. **关键词分析** - 识别各年度的核心主题
4. **可视化展示** - 直观展现数据特征
5. **自我激励** - 基于数据的正向反馈

这些分析不仅展现了写作的数量特征，更重要的是体现了思维的深度和广度。
继续保持这份对文字的热爱和对思考的坚持！ 🚀✨